<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/Colab/Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [392]:
from collections import defaultdict
import pandas as pd
import numpy as np
from math import log, e
import csv

In [408]:
# Print out the 'list' to the 'name.csv' file 

# def printOut(name, list):
#     with open(name + '.csv', 'w', newline='') as csv_1:
#         csv_out = csv.writer(csv_1)
#         csv_out.writerows([list[index]] for index in range(0, len(list)))

In [409]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [410]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [411]:
# List of the positions of nodes
positions = positionRead("Network/Brain_data/Brain1Positions")

# List of how nodes connected to each other
connections = connectionRead("Network/Brain_data/Brain1Connections")

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [412]:
# Calculate distance between 2 points in Euclidean space
# Positions should be 3D (X,Y,Z)

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [413]:
# Entropy by numpy library

def entropy(labels, base=None):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0
    
    value, counts = np.unique(labels, return_counts=True)
    probs = counts/n_labels
    n_classes = np.count_nonzero(probs)
    
    if n_classes <= 1:
        return 0
    
    ent = 0.
    
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)
        
    return ent

In [414]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p): 
    neighbours = []
    
    for i in range(len(positions[0])):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [415]:
positions

,0,1,2
0,86.828723,55.964199,26.615948
1,68.934166,51.139614,32.673099
2,99.234568,29.370370,38.259259
3,101.797312,49.265398,28.711646
4,65.881081,61.874595,43.203784
...,...,...,...
78,125.030392,96.151471,28.293137
79,114.409524,80.682993,26.676190
80,130.959165,111.471842,12.879515
81,128.398802,94.169108,12.902796


In [416]:
connections

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
79,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,1,1
80,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,0,1,1
81,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [417]:
def toDictionary(positions, connections):
  edges = []
  for i in range(len(connections[0])):
    for j in range(len(connections[0])):
      if i!=j and i<j and connections[i][j]==1:
        edges.append((i,j,Euclidean_dist(i,j)))
  return edges

In [424]:
class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        self.shortest_path is a dict of node - full distance pairs
        e.g init node is 1 -> {8: (4, 60.511292298086815)} means from init node 1 through node 4 to node 8, the full distance between node 1 and node 8 is 60.51...
        self.node2node_paths is a dict of node and node to node ditances
        e.g init node is 1 -> {8: (4, 45.166701725970064)} means from node 4 to node 8 the distance is 45.166...
        """
        self.edges = defaultdict(list)
        self.weights = {}
        self.shortest_paths = {}
        self.node2node_paths = {}
#------------------------------------------------------------------------------------------------------------------------------------    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight
#--------------------------------------------------------------------------------------------------------------------------------------
    def dijsktra(self, initial, end):
      # shortest paths is a dict of nodes
      # whose value is a tuple of (previous node, weight)
      current_node = initial
      visited = set()
      self.shortest_paths = {initial: (None, 0)}
      self.node2node_paths = {initial: (None, 0)}
    
      while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = self.shortest_paths[current_node][1]

        for next_node in destinations:
            nodeDist = graph.weights[(current_node, next_node)]
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in self.shortest_paths:
                self.shortest_paths[next_node] = (current_node, weight)
                self.node2node_paths[next_node] = (current_node, nodeDist)
            else:
                current_shortest_weight = self.shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    self.shortest_paths[next_node] = (current_node, weight)
                    self.node2node_paths[next_node] = (current_node, nodeDist)
        
        next_destinations = {node: self.shortest_paths[node] for node in self.shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
#-----------------------------------------------------------------------------------------------------------------------------------------------
    def BFS_SP(self, initial, end):
      explored = []
     
      # Queue for traversing the
      # graph in the BFS
      queue = [[initial]]
     
      # If the desired node is
      # reached
      if initial == end:
        print("Same Node")
        return
     
      # Loop to traverse the graph
      # with the help of the queue
      while queue:
        path = queue.pop(0)
        node = path[-1]
         
        # Condition to check if the
        # current node is not visited
        if node not in explored:
            neighbours = self.edges[node]
             
            # Loop to iterate over the
            # neighbours of the node
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                 
                # Condition to check if the
                # neighbour node is the goal
                if neighbour == end:
                    return new_path
            explored.append(node)
 
      # Condition when the nodes
      # are not connected
      print("Path doesn't exist.")
      return
#------------------------------------------------------------------------------------------------------------------------------------    
    def findAllBFS(self, initial):
      paths = []
      for i in range(len(connections[0])):
        if i!=initial:
          paths.append(self.BFS_SP(initial, i))
      return path
#------------------------------------------------------------------------------------------------------------------------------------    
    def findAllDijkstra(self, initial):
      for i in range(len(connections)):
        if i!=initial:
          self.dijsktra(initial, i)
          if(len(self.shortest_paths)==len(connections[0])):
            break;
#------------------------------------------------------------------------------------------------------------------------------------    
    def shortestPath(self, initial, end):
      # Work back through destinations in shortest path
      current_node = end
      path = []
      while current_node is not None:
        path.append([current_node, self.node2node_paths[current_node][1]])
        next_node = self.shortest_paths[current_node][0]
        current_node = next_node
      # Reverse path
      path = path[::-1]
      return path

In [425]:
# Initialize graph

graph = Graph()
edges = toDictionary(positions, connections)

for edge in edges:
    graph.add_edge(*edge)

TypeError: ignored

In [426]:
# Calculate all shortest paths (Dijkstra)

paths = []

for i in range(len(connections[0])):
  graph.findAllDijkstra(i)
  for j in range(len(connections[0])):
    paths.append(graph.shortestPath(i,j))

In [421]:
nodes = []
dist = []

for i in paths:
  path_nodes = []
  path_dist = []
  for j in range(len(i)-1):
    path_nodes.append(i[j+1][0])
    path_dist.append(i[j+1][1])
  nodes.append(path_nodes)
  dist.append(path_dist)

In [422]:
nodes

[[],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [5, 9],
 [35, 10],
 [3, 11],
 [3, 12],
 [3, 13],
 [40, 39, 14],
 [5, 15],
 [33, 16],
 [17],
 [18],
 [36, 19],
 [37, 20],
 [37, 21],
 [22],
 [23],
 [24],
 [40, 25],
 [27, 26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [33, 32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [40, 39],
 [40],
 [3, 44, 41],
 [3, 44, 41, 42],
 [3, 43],
 [3, 44],
 [3, 44, 41, 45],
 [38, 76, 46],
 [3, 47],
 [3, 48],
 [35, 49],
 [38, 50],
 [38, 76, 51],
 [3, 52],
 [3, 11, 53],
 [38, 76, 54],
 [35, 75, 55],
 [38, 76, 56],
 [38, 78, 57],
 [37, 58],
 [38, 78, 59],
 [34, 60],
 [37, 61],
 [38, 78, 62],
 [82, 63],
 [82, 64],
 [38, 81, 65],
 [38, 81, 67, 66],
 [38, 81, 67],
 [38, 81, 68],
 [38, 78, 69],
 [38, 78, 70],
 [38, 77, 74, 71],
 [38, 77, 72],
 [38, 78, 73],
 [38, 77, 74],
 [35, 75],
 [38, 76],
 [38, 77],
 [38, 78],
 [38, 79],
 [38, 79, 80],
 [38, 81],
 [82],
 [0],
 [],
 [0, 2],
 [0, 3],
 [4],
 [5],
 [6],
 [7],
 [4, 8],
 [5, 9],
 [35, 10],
 [0, 3, 11],
 [7, 12],
 [7

In [387]:
ent = []

for i in dist:
  ent.append(entropy(i))

In [388]:
ent_nodes = []

for i in nodes:
  ent_nodes.append(entropy(i))

In [389]:
# printOut('Network/Data/Centroid1/entDistShortest1', ent)
# printOut('Network/Data/Centroid1/entNodesShortest1', ent_nodes)